In [0]:
import os
import cv2 #conda install -c conda-forge opencv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

from shutil import copy

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, Dropout, Flatten, concatenate, Dense, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

#tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
GOOGLE_DIR = 'gdrive/My Drive/flowers/'
IMG_DIR = GOOGLE_DIR + 'flowers/'
TRAIN_DATA_DIR = GOOGLE_DIR + 'train/'
TEST_DATA_DIR = GOOGLE_DIR + 'test/'
IMG_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 100

In [0]:
x_ = list()
y = list()

nb = 0
for x in os.listdir(IMG_DIR):
    for i in os.listdir(IMG_DIR + x):
        path = (IMG_DIR + x + '/' + i)
        img = plt.imread(path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        x_.append(img)
        y.append(nb)
    nb += 1

In [0]:
x_ = np.array(x_)
y = to_categorical(y, num_classes=5)
print(x_.shape, y.shape)

(4331, 128, 128, 3) (4331, 5, 5)


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_, y, test_size=0.20, random_state=42)


In [0]:
model = Sequential()

model.add(Conv2D(256, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 128, 128, 256)     7168      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 64, 64, 256)       0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 64, 64, 256)       1024      
_________________________________________________________________
dropout_46 (Dropout)         (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 64, 64, 128)       295040    
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 32, 32, 128)       0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 32, 32, 128)     

In [0]:
padding = 'valid'
img_input = Input(shape=(128, 128, 3))
conv_1 = Conv2D(64, (5, 5), strides=(2, 2), padding=padding, activation='relu', name='conv_1')(img_input)
drop_1 = Dropout(0.2)(conv_1)
maxpool_1 = MaxPooling2D((2, 2), strides=(2,2))(drop_1)
x = BatchNormalization()(maxpool_1)

# FEAT-EX1
conv_2a = Conv2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_2a')(x)
conv_2b = Conv2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_2b')(conv_2a)
drop_2 = Dropout(0.2)(conv_2b)

maxpool_2a = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_2a')(x)
conv_2c = Conv2D(64, (1, 1), strides=(1,1), name='conv_2c')(maxpool_2a)
drop_3 = Dropout(0.2)(conv_2c)

concat_1 = concatenate(inputs=[drop_2, drop_3], axis=3, name='concat2')
maxpool_2b = MaxPooling2D((3,3), strides=(2,2), padding=padding, name='maxpool_2b')(concat_1)

# FEAT-EX2
conv_3a = Conv2D(96, (1, 1), strides=(1,1), activation='relu', padding=padding, name='conv_3a')(maxpool_2b)
conv_3b = Conv2D(208, (3, 3), strides=(1,1), activation='relu', padding=padding, name='conv_3b')(conv_3a)

maxpool_3a = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3a')(maxpool_2b)
conv_3c = Conv2D(64, (1, 1), strides=(1,1), name='conv_3c')(maxpool_3a)
drop_4 = Dropout(0.3)(conv_3c)

concat_3 = concatenate(inputs=[conv_3b, conv_3c], axis=3, name='concat3')
maxpool_3b = MaxPooling2D((3,3), strides=(1,1), padding=padding, name='maxpool_3b')(concat_3)

# FINAL LAYERS
net = Flatten()(maxpool_3b)
output = Dense(5, activation='softmax', name='predictions')(net)

# Create model.
model = Model(img_input, output, name='deXpression')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

NameError: ignored

In [0]:
  datagen = ImageDataGenerator(
      featurewise_center=False,  # set input mean to 0 over the dataset
      samplewise_center=False,  # set each sample mean to 0
      featurewise_std_normalization=False,  # divide inputs by std of the dataset
      samplewise_std_normalization=False,  # divide each input by its std
      rotation_range=60,  # randomly rotate images in the range (60, 0 to 180)
      zoom_range=0.1,  # Randomly zoom image
      width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
      height_shift_range=0.1,
      shear_range=0.1,
      fill_mode="reflect"
  )
  datagen.fit(x_train)

In [0]:
  earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
  cp_callback = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', mode='min',
                                save_weights_only=False, verbose=1, save_best_only=True, period=1)

In [0]:
  history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                                epochs=EPOCHS, validation_data=(x_test, y_test),
                                steps_per_epoch=x_train.shape[0] // BATCH_SIZE,
                                callbacks=[earlyStopping, cp_callback]
                                )

NameError: ignored

In [0]:
  # plot loss
  plt.title('Cross Entropy Loss')
  plt.plot(history.history['loss'], color='blue', label='train')
  plt.plot(history.history['val_loss'], color='orange', label='test')
  plt.savefig('crossentropy_loss.png')
  plt.close()
  # plot accuracy
  plt.title('Classification Accuracy')
  plt.plot(history.history['accuracy'], color='blue', label='train')
  plt.plot(history.history['val_accuracy'], color='orange', label='test')
  plt.savefig('accuracy.png')
  plt.close()